In [23]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [3]:
df = pd.read_csv('trained_model/preds/tweets_preds.csv')

# Raw Text

In [12]:
text = ' '.join(df.text.values)
text = text.lower()

In [14]:
# create mapping of unique chars to integers
chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [18]:
chars[:10]

['\n', '\r', ' ', '!', '"', '#', '$', '%', '&', "'"]

## Text Characteristics

In [21]:
n_chars = len(text)
n_vocab = len(chars)
print( "Total Characters: ", n_chars)
print( "Total Vocab: ", n_vocab)

Total Characters:  1527438
Total Vocab:  1629


## Prepare the dataset of input to output pairs encoded as integers

In [22]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = text[i:i + seq_length]
	seq_out = text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print( "Total Patterns: ", n_patterns)

Total Patterns:  1527338


## Reshape X to be [samples, time steps, features]

In [24]:
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

## Define the LSTM model - Choose among single and deep, comment one

In [25]:
# Single Model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# Deeper - 2 layers (better results, more computing time)
"""model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')"""

## Define the checkpoint
Because of the slowness and because of our optimization requirements, we will use model checkpointing to record all of the network weights to file each time an improvement in loss is observed at the end of the epoch. We will use the best set of weights (lowest loss) to instantiate our generative model in the next section.

In [26]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

## Fit model to data

In [28]:
# Lesser epochs lesser time worst results
#model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

# More epochs, more time, best results
#model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

After running the example, you should have a number of weight checkpoint files in the local directory.

You can delete them all except the one with the smallest loss value. For example, when I ran this example, below was the checkpoint with the smallest loss that I achieved.

In [29]:
# Best checkpoint name here

## Generating text

Generating text using the trained LSTM network is relatively straightforward.

Firstly, we load the data and define the network in exactly the same way, except the network weights are loaded from a checkpoint file and the network does not need to be trained.

In [ ]:
# load the network weights
filename = "weights-improvement-19-1.9435.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [30]:
# Reverse mapping
int_to_char = dict((i, c) for i, c in enumerate(chars))

## Make predictions

The simplest way to use the Keras LSTM model to make predictions is to first start off with a seed sequence as input, generate the next character then update the seed sequence to add the generated character on the end and trim off the first character. This process is repeated for as long as we want to predict new characters (e.g. a sequence of 1,000 characters in length).

We can pick a random input pattern as our seed sequence, then print generated characters as we generate them.

In [ ]:
import sys
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print( "Seed:")
print( "\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print( "\nDone.")

## 10 Extension Ideas to Improve the Model


Predict fewer than 1,000 characters as output for a given seed.

Remove all punctuation from the source text, and therefore from the models’ vocabulary.

Try a one hot encoded for the input sequences.

Train the model on padded sentences rather than random sequences of characters.

Increase the number of training epochs to 100 or many hundreds.

Add dropout to the visible input layer and consider tuning the dropout percentage.

Tune the batch size, try a batch size of 1 as a (very slow) baseline and larger sizes from there.

Add more memory units to the layers and/or more layers.

Experiment with scale factors (temperature) when interpreting the prediction probabilities.

Change the LSTM layers to be “stateful” to maintain state across batches.